In [1]:
import os
import re
import json
import time
import hashlib
import random
from bs4 import BeautifulSoup
from urllib.request import urlopen
import ssl

In [2]:
IMAGEDIR='images'
PROFILES='real'

iurlrx = re.compile('.* background-image: url\(([^\)]+)\)')

remap = {'I am' : 'gender',
         'Age' : 'age',
         'City' : 'location',
         'Marital status' : 'status',
         'Username' : 'username',
         'Ethnicity' : 'ethnicity',
         'Occupation' : 'occupation',
         'About me' : 'description',
         'My match\'s age' : 'match_age',
         'Children' : 'children',
         'Sexual Orientation' : 'orientation',
         'Religion' : 'religion',
         'Do you smoke' : 'smoking',
         'Do you drink' : 'drinking',
         'Here for' : 'intent'}

def save_image(url):
    """ Take a URL, generate a unique filename, save 
        the image to said file and return the filename."""
    ext = url.split('.')[-1]
    filename = IMAGEDIR+os.sep+hashlib.md5(url.encode('utf-8')).hexdigest()+'.'+ext
    if os.path.exists(filename):
        return filename
    try:
        content = urlopen(url).read()
        f = open(filename,'wb') 
        f.write(content)
        f.close()
    except e:
        print(e)
        return None
    return filename 




- Generate a list of urls within the scrape profiles 
- then go into each url to get the profile


In [3]:
def scrape_profile(inhandle):
  """Scrape an input scamdiggers page for the profile content
  of the scammer. """
  #Read file
  html = inhandle.read()
  soup = BeautifulSoup(html, 'html.parser')

  #pfnode = soup.find('div', {'class':'profile-BASE_CMP_UserViewWidget'})
  #avnode = soup.find(id='avatar_console_image')
  target_divs = soup.find_all('div', class_='ow_user_list_item clearfix ow_item_set3')
  # Initialize an empty list to store unique links
  unique_links = []

  # Iterate over each div element
  for div in target_divs:
      # Find all 'a' tags with 'href' attribute and extract the 'href' value
    links = [a['href'] for a in div.find_all('a', href=True)]
    # Add links to the unique_links list
    unique_links.extend(links)

  # Get unique links
  unique_links = list(set(unique_links))


  for link in unique_links:
    urltemp = urlopen(link,context=ssl._create_unverified_context())
    htmltemp = urltemp.read()
    souptemp = BeautifulSoup(htmltemp, 'html.parser')
    #Pull the provided profile data out.
    rows = souptemp.findAll('tr')
    labels = {}
    for row in rows:
      lab = row.find('td',{'class':'ow_label'})
      val = row.find('td',{'class':'ow_value'})
      if lab:
        labels[lab.get_text()] = val.get_text().strip()

    profile = {}

    #Populate our own profile structure.
    for lab in remap:
      if lab in labels:
        profile[remap[lab]] = labels[lab]
      else:
        profile[remap[lab]] = "-"
  
    #Tweak for consistency.
    profile['gender'] = profile['gender'].lower()
  
    #Extract avatar image
    #img = iurlrx.match(avnode.attrs['style']).group(1)
    #profile['images'] = [save_image(img)]

    #Save output
    print(profile)
    outfile=outfile=profile['username']+'.json'
    json.dump(profile, open(outfile,'w'))





In [4]:
def enumerate_profiles(inhandle):
  """ Extract all the profile page links from
  this index page. """
  html = inhandle.read()
  soup = BeautifulSoup(html, 'html.parser')
  
  urls = [ node.find('a')['href'] for node in soup.findAll('div',  {'class':'ow_user_list_data'})]
  return urls




In [5]:
def scrape():
  """ Harvest profiles from every third page from the site. """
  urls = []
  urlstr="http://datingnmore.com/site/users/latest?page={}" 

  print("Begin URL harvesting.")

  #For every third page (sample size calculated to finish overnight). 
  # edit the number of pages u want 
  for i in range(500,1000,3):
    url = urlstr.format(i)
    jitter = random.choice([0,1])
    try:
      urlhandle = urlopen(url)
      print(urlhandle)
      scrape_profile(urlhandle)
      
    except Exception as e:
      print("Exception when handling {}".format(url))
      print(e)
      break

  print("Harvesting complete. {} URLs to scrape.".format(len(urls)))
      
 
 
  print("Scraping complete.")


scrape()

Begin URL harvesting.
{'gender': 'male', 'age': '46 y.o.', 'location': 'Herriman, UT 84096, USA', 'status': 'divorced', 'username': 'josem357', 'ethnicity': 'white', 'occupation': '-', 'description': 'Me considero trabajador, atento, divertido busco pasarla bien y divertirme me esmero en divertir y hacer pasar bien a mi pareja.', 'match_age': 'from 30 to 48', 'children': 'more than 2 living elsewhere', 'orientation': 'Straight', 'religion': 'Christian', 'smoking': 'non-smoker', 'drinking': 'social drinker', 'intent': 'Fun, Serious Relationship, Marriage'}
{'gender': 'male', 'age': '35 y.o.', 'location': 'Cape Town, South Africa', 'status': 'single', 'username': 'Jaejay', 'ethnicity': 'mixed', 'occupation': 'Handyman', 'description': '-', 'match_age': 'from 36 to 80', 'children': '1-2 living elsewhere', 'orientation': 'Straight', 'religion': 'Christian', 'smoking': 'social smoker', 'drinking': 'social drinker', 'intent': 'Fun'}
{'gender': 'female', 'age': '22 y.o.', 'location': 'San Sal